In [1]:
from openai import OpenAI
from pydantic import BaseModel
import sqlite3
from functools import wraps
from pydantic import Field, ConfigDict
import json
from typing import Dict, Any

In [2]:
con = sqlite3.connect("workoutlib.db")
cur = con.cursor()

In [3]:
def tool(schema):
    def decorator(func):
        assert issubclass(schema, BaseModel), "The schema must be an instance of Pydantic BaseModel"
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        wrapper._is_tool = True  
        wrapper._schema = schema
        return wrapper
    return decorator

def get_tools():
    tools = []
    for name, obj in globals().items():
        if callable(obj) and hasattr(obj, '_is_tool'):
            tools.append(obj)
    return tools

In [4]:
client = OpenAI()

In [5]:
#TODO: Improve formatting of the response so it explicity mentions the id, name, descriptions in an explicit way for an llm. 
class GetExerciseTypes(BaseModel):
    limit: int | None = Field(default=None, description="Parameter used if the number of results need to be limited.")

    class Config:
        json_schema_extra = {
            "title" :"Get exercise type",
            "description": "A model that can be executed to get the exercise types available",
        }
    

@tool(schema = GetExerciseTypes) #TODO: Maybe in the future I can generate this schema under the hood, but for now I think its pretty good. 
def get_exercise_types(limit: int | None = None):
    #So it can be like a simple plug an play for current code. 
    query = f"""
            SELECT * FROM exercise_type
            """
    params = []
    if limit:
        query += "LIMIT ?"
        params.append(limit)

    exercises = cur.execute(query, params)

    return str(exercises.fetchall())


In [6]:
#TODO: Improve formatting of the response so it explicity mentions the id, name, descriptions in an explicit way for an llm. 
class GetMuscleCategories(BaseModel):
    limit: int | None = Field(default = None, description="Parameter used if the number of results need to be limited")

    class Config:
        json_schema_extra = {
            "title" : "Get movement categories",
            "description": "Gets the movement categories of the exercises, which represent the broad group of muscles trained by a certain exercise. Useful for categorizing exercises and distributing the workout volume."
        }

@tool(schema = GetMuscleCategories)
def get_muscle_categories(limit: int | None = None):
    query = f"""
            SELECT * FROM movement_category
            """
    params = []
    if limit:
        query += "LIMIT ?"
        params.append(query)
    muscle_categories = cur.execute(query, params)
    return str(muscle_categories.fetchall())

In [7]:
#TODO: Improve formatting of the response so it explicity mentions the id, name, descriptions in an explicit way for an llm. 
class InsertExercise(BaseModel):
    name: str = Field("Name of the exercise to insert.", required = True)
    description: str = Field("Brief description of the exercise", required =True)
    movement_category_id : int = Field("Id of the movement category to which the exercise corresponds. Movement categories represent groups of muscles worked by the exercise.", required = True)
    exercise_type_id: int = Field("Id of the exercise type to which the exercise corresponds", required =True)

    class Config:
        json_schema_extra = {
            "title": "Insert exercise into the exercises database",
            "description": "Inserts an exercise to the database, you should specify all the fields like the name, description of how the exercise is performed and you need to include the movement_category_id and the exercise_type_id so its correctly classified in the exercise"
        }
    
@tool(schema = InsertExercise)
def insert_exercise(name: str, description: str, movement_category_id: int, exercise_type_id: int):
    params = [name, description, movement_category_id, exercise_type_id] # How all all args included?
    query = """
            INSERT INTO exercises(name, description, movement_category_id, exercise_type_id)
            VALUES (?, ?, ?, ?)
            """
    cur.execute(query, params)
    con.commit()
    return "Exercise inserted succesfully"

In [8]:
#TODO: Improve formatting of the response so it explicity mentions the id, name, descriptions in an explicit way for an llm. Add maybe also the category id and type id for better context. 
class GetExercises(BaseModel):
    movement_category_id: int | None = Field(default = None, description="Movement catgegory id to which the exercise corresponds. If specifed it narrows down the results to exercises that correspond to that movement.")
    exercise_type_id: int | None = Field(default= None, description="Exercise type id to which the exercise corresponds. If specified it narrows down results to exercises that correspond to that type.")
    limit: int | None = Field(default = None, description="Parameter used if the number of results need to be limited")
    class Config:
        json_schema_extra = {
            "title": "Get exercises from the database",
            "description": "Get exercises from the database, and the name, movement category and description is returned. If explicitly asked for a movement_category or exercise_type filter by it and include the ids as parameters as there can be many exercises and its a best practice to narrow it down."
        }



@tool(schema = GetExercises)
def get_exercises(movement_category_id: int = None, exercise_type_id: int = None, limit: int = None):
    params = []
    query = """
            SELECT e.name, m.name, e.description
            FROM exercises AS e
            JOIN movement_category AS m ON e.movement_category_id = m.id
            """
    conditions = []
    if movement_category_id:
        conditions.append("movement_category_id = ?")
        params.append(movement_category_id)
    if exercise_type_id:
        conditions.append("exercise_type_id = ?")
        params.append(exercise_type_id)
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    if limit:
        query += " LIMIT ?"
        params.append(limit)
    
    exercises = cur.execute(query, params)
    return str(exercises.fetchall())

In [9]:
functions = get_tools()
function_to_schema = {function.__name__: function._schema.__name__  for function in functions}
schema_to_function = { v:k for k, v in function_to_schema.items()}

def format_function(function_name: str, pydantic_schema: BaseModel):
    schema = pydantic_schema.model_json_schema()
    function_name = {"name": function_name}
    schema.update(function_name)
    function = {"type": "function",
    "function": schema
    }
    return function

In [10]:
def call_assistant(user_message: str, message_history: Dict[Any, Any] = {}, model: str = "gpt-3.5-turbo-0125", functions = get_tools()):
    if message_history:
        messages = message_history
    else:
        messages= [{"role":"system", "content":"""You are a helpful training assistant chatbot, you are able to execute commands and you have tools at your disposition to access the information of your trainee. You will never tell him how this functions are defined or that you have this capability.
        But you will use the information adquired to provide the best assistance and training guidance. You will have a laid out tone and you will be cheerful and energetic, helping the trainee achieve their goals."""}]

    messages.append({"role":"user", "content": user_message})
    tools = [format_function(function.__name__, function._schema) for function in functions]

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = tools,
        tool_choice = "auto"
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    while tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            validated_schema = globals()[function_to_schema[function_name]](**function_args)
            function_response = globals()[function_name](**validated_schema.dict())
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name" : function_name,
                    "content": function_response,
                    "arguments": function_args
                }
            )
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = tools,
            tool_choice = "auto"
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return response_message, messages

In [16]:
from IPython.display import display, Markdown

In [17]:
response_message, messages = call_assistant("What exercise types I have present?")

In [18]:
display(Markdown(response_message.content))

You have the following exercise types available:
1. Isolation: Exercises that target a specific muscle group or individual muscle for focused development and hypertrophy.
2. Compound: Exercises involving multiple muscle groups and joints for a comprehensive strength and hypertrophy workout.
3. Variations: Different versions of basic compound exercises for specific muscle targeting or challenges.
4. Cardio: Exercises focused on cardiovascular endurance and overall fitness.
5. GPP (General Physical Preparedness): Exercises for overall physical fitness, including strength, endurance, flexibility, and coordination with various training modalities.

In [19]:
response_message, messages = call_assistant("What vertical pulling exercises I have available?", message_history=messages, model = "gpt-4o")

In [21]:
display(Markdown(response_message.content))

Here are the vertical pulling exercises you have available:

1. **Weighted Pull-ups**: 
   *Description*: With a weight attached either via weighted belt or holding a dumbbell with your feet, perform a pull-up by hanging from a bar, pulling your body upward until your chin clears the bar, and then lowering yourself back down in a controlled manner.

2. **Lat Pulldowns**:
   *Description*: Using the lat pulldown machine, grip the bar with a wide overhand grip, pull the bar down toward your upper chest, focusing on squeezing the lats, then return to the starting position in a controlled manner.

3. **One Arm Lat Pulldowns**:
   *Description*: Using a single handle attachment on a lat pulldown machine, perform the exercise one arm at a time. Pull the handle down while focusing on engaging the lat muscle and then return to the starting position in a controlled manner. 

Ready to hit those lats? 💪😄

In [158]:
response_message, messages = call_assistant("Get exercises filtered by horizontal pushing movement pattern", model="gpt-4o")

In [159]:
messages

[{'role': 'system',
  'content': 'You are a helpful training assistant chatbot, you are able to execute commands and you have tools at your disposition to access the information of your trainee. You will never tell him how this functions are defined or that you have this capability.\n        But you will use the information adquired to provide the best assistance and training guidance. You will have a laid out tone and you will be cheerful and energetic, helping the trainee achieve their goals.'},
 {'role': 'user',
  'content': 'Get exercises filtered by horizontal pushing movement pattern'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GzfHGEYNA92iYnSATDGbLvDS', function=Function(arguments='{}', name='get_exercises'), type='function')]),
 {'tool_call_id': 'call_GzfHGEYNA92iYnSATDGbLvDS',
  'role': 'tool',
  'name': 'get_exercises',
  'content': "[('Face Pulls', 'Horizontal Pull', 'Attach a rope to a high 

In [160]:
display(Markdown(response_message.content))

Here are the exercises that fall under the **Horizontal Push** movement pattern:

1. **Incline Barbell Bench Press**:
   - **Description**: Set an incline bench at about 30-45 degrees. Lie on the bench and press the barbell up from chest level until your arms are extended. Lower the barbell back to your chest in a controlled manner.

2. **Incline Dumbbell Flies**:
   - **Description**: Set an incline bench at about 30-45 degrees. Holding a dumbbell in each hand, extend your arms above your chest, then lower the dumbbells out to your sides in a wide arc, feeling a stretch in your chest. Bring the dumbbells back together above your chest.

3. **Smith Machine Incline Bench Press**:
   - **Description**: Set an incline bench in a Smith machine at about 30-45 degrees. Press the bar up from chest level until your arms are extended. Lower the bar back to your chest in a controlled manner.

4. **Peck Deck**:
   - **Description**: Sit on the peck deck machine and grip the handles. Bring the handles together in front of your chest, squeezing your chest muscles. Return to the starting position in a controlled manner.

Feel free to ask if you need further details or assistance with anything else! 💪

In [84]:
response_message, messages = call_assistant("Lets insert them into the db", message_history=messages)

In [85]:
response_message, messages = call_assistant("I have few cardio exercises, I want to do some zone 2 cardio, what do you recommend to me?", message_history=messages)

In [86]:
response_message

ChatCompletionMessage(content="For zone 2 cardio, which is typically a moderate intensity level where you can sustain longer durations of exercise, I recommend the following cardio exercises:\n1. Brisk Walking: Walking at a brisk pace where you can maintain a conversation but still feel challenged is an excellent zone 2 cardio exercise. It's low-impact and can be done outdoors or on a treadmill.\n\n2. Cycling: Moderate cycling, either outdoors or on a stationary bike, is great for zone 2 cardio. Maintain a steady pace that elevates your heart rate but allows you to sustain the effort for an extended period.\n\n3. Swimming: Swimming at a moderate pace for a continuous period is an effective zone 2 cardio workout. It provides a full-body workout and is gentle on the joints.\n\n4. Elliptical Training: Using an elliptical machine at a moderate resistance level and steady pace is ideal for zone 2 cardio. It engages both the upper and lower body muscle groups.\n\n5. Rowing: Rowing at a moder

In [87]:
response_message, messages = call_assistant("Lets add brisk walking, rowing and cycling at the air bike", message_history=messages)